In [32]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.cluster import KMeans

# 1. Đọc dữ liệu và xóa khoảng trắng thừa
df = pd.read_csv("data_students.csv")

# Lấy các cột dạng chuỗi ngoại trừ 'final_result'
cols_to_strip = df.select_dtypes(include='object').columns.drop('final_result', errors='ignore')

# Xoá khoảng trắng cuối ở các cột cần xử lý (trừ 'final_result')
for col in cols_to_strip:
    df[col] = df[col].str.strip()
df.dropna(inplace=True)
df.to_csv("data_students.csv",index=False)


In [33]:
# 2. Chọn các đặc trưng dùng để phân cụm
features = ['gender', 'region', 'entrance_result', 'num_of_prev_attempts', 'studied_credits', 'disability']

# 3. Mã hóa nhãn (Label Encoding cho cột dạng text)
df_encoded = df.copy()
le_dict = {}

for col in features:
    if df_encoded[col].dtype == 'object':
        le = LabelEncoder()
        df_encoded[col] = le.fit_transform(df_encoded[col])
        le_dict[col] = le  # Lưu lại encoder nếu cần dùng sau

# 4. Chuẩn hóa dữ liệu
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df_encoded[features])

# 5. Phân cụm KMeans
kmeans = KMeans(n_clusters=3, random_state=42)
df_encoded['cluster'] = kmeans.fit_predict(X_scaled)

# 6. Gán nhãn A/B/C theo cụm
# → Tính trung bình số tín chỉ đã học của từng cụm để sắp xếp
cluster_mean = df_encoded.groupby('cluster')['studied_credits'].mean().sort_values(ascending=False)
label_map = {cluster: label for cluster, label in zip(cluster_mean.index, ['A', 'B', 'C'])}
df_encoded['cluster_label'] = df_encoded['cluster'].map(label_map)

# 7. Thêm kết quả vào file
df['cluster'] = df_encoded['cluster']
df['cluster_label'] = df_encoded['cluster_label']

# 8. Ghi file kết quả
df.to_csv("student_clustered.csv", index=False)

print("✅ Xử lý và phân cụm hoàn tất! File lưu tại: student_clustered.csv")
df


✅ Xử lý và phân cụm hoàn tất! File lưu tại: student_clustered.csv


,id_student,gender,region,entrance_result,num_of_prev_attempts,studied_credits,disability,final_result,cluster,cluster_label
0,11391.0,M,Addis Ababa,HE Qualification,0.0,240.0,N,Pass,1,B
1,28400.0,F,Afar Region,HE Qualification,0.0,60.0,N,Pass,2,C
2,30268.0,F,Amhara Region,A Level or Equivalent,0.0,60.0,Y,Withdrawn,2,C
3,31604.0,F,Oromia Region,A Level or Equivalent,0.0,60.0,N,Pass,2,C
4,32885.0,F,South Region,Lower Than A Level,0.0,60.0,N,Pass,2,C
...,...,...,...,...,...,...,...,...,...,...
33276,2640965.0,F,Tigrai Region,Lower Than A Level,0.0,30.0,N,Fail,2,C
33277,2645731.0,F,Addis Ababa,Lower Than A Level,0.0,30.0,N,Distinction,2,C
33278,2648187.0,F,South Region,A Level or Equivalent,0.0,30.0,Y,Pass,2,C
33279,2679821.0,F,Oromia Region,Lower Than A Level,0.0,30.0,N,Withdrawn,2,C


In [34]:
import pandas as pd

# Giả sử df là DataFrame đã có cột 'final_result' và 'cluster'
# Bước 1: Tính bảng phân phối phần trăm
table = pd.crosstab(df['final_result'], df['cluster'], normalize='columns') * 100

# Bước 2: Đổi tên cột cụm để đúng như bảng mô tả
table.columns = ['C0 (grades A) (%)', 'C1 (grades B) (%)', 'C2 (grades C) (%)']

# Bước 3: Reset index và in bảng
table = table.reset_index()
print(table.to_string(index=False))


final_result  C0 (grades A) (%)  C1 (grades B) (%)  C2 (grades C) (%)
 Distinction           3.991396           9.830833          10.334275
        Fail          31.572658          20.663900          19.569994
        Pass          29.063098          38.314714          40.356246
    Withdraw           0.000000           0.000000           0.007847
   Withdrawn          35.372849          31.190552          29.731638
